# Course Recommender: Collaborative-Filtering Methods

This project implements and deploys an AI course Recommender System using [Streamlit](https://streamlit.io/). It was inspired by the the [IBM Machine Learning Professional Certificate](https://www.coursera.org/professional-certificates/ibm-machine-learning) offered by IBM & Coursera. In the last course/module of the Specialization, Machine Learning Capstone, a similar application is built; check my [class notes](https://github.com/mxagar/machine_learning_ibm/tree/main/06_Capstone_Project/06_Capstone_Recommender_System.md) for more information.

This notebook researches and implements different **Collaborative-Filtering** recommender systems. You can open it in Google Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mxagar/course_recommender_streamlit/blob/main/notebooks/04_Collaborative_RecSys.ipynb)

The following is implemented in this notebook:
  
  - A dense user-items ratings table is converted to a sparse table.
  - Item-based and user-based k-NN search is applied to find the closest items/users given their similarity and perform a prediction with a weighted sum. This is done with the [surprise](https://surprise.readthedocs.io/en/stable/index.html) library and manually.
  - Non-Negative Matrix Factorization (NMF) is used to decompose a ratings table into lower rank matrices which encode latent features. This is done with the [surprise](https://surprise.readthedocs.io/en/stable/index.html) library and with Scikit-Learn.

Table of contents:

- [Part 1: k-NN](#Part-1:-k-NN)
    - [1. Convert Dense Ratings Matrix to Sparse User-Item Matrix](#1.-Convert-Dense-Ratings-Matrix-to-Sparse-User-Item-Matrix)
    - [2. Item-Based Collaborative Filtering: k-NN with Suprise Library](#2.-Item-Based-Collaborative-Filtering:-k-NN-with-Suprise-Library)
    - [3. User-Based Collaborative Filtering: k-NN Manually](#3.-User-Based-Collaborative-Filtering:-k-NN-Manually)
- [Part 2: Non-Negative Matrix Factorization (NMF)](#Part-2:-Non-Negative-Matrix-Factorization-(NMF))
    - [1. NMF with the Suprise Library](#1.-NMF-with-the-Suprise-Library)
    - [2. NMF with Scikit-Learn](#2.-NMF-with-Scikit-Learn)

# Basic Definitions

**User-based** collaborative filtering: user similarity matrix is computed (user row against user row with cosine similarity) and then k most similar users are used to estimate ratings as weighted sums:

$$\hat{r}_{ui} = \frac{
\sum\limits_{v \in N^k_i(u)} \text{similarity}(u, v) \cdot r_{vi}}
{\sum\limits_{v \in N^k_i(u)} \text{similarity}(u, v)}$$

**Item-based** collaborative filtering: item similarity matrix is computed (item column against item column with cosine similarity) and then k most similar items are used to estimate ratings as weighted sums:

$$\hat{r}_{ui} = \frac{
\sum\limits_{j \in N^k_u(i)} \text{similarity}(i, j) \cdot r_{uj}}
{\sum\limits_{j \in N^k_u(i)} \text{similarity}(i, j)}$$


In [4]:
import numpy as np
import pandas as pd
import math

# Part 1: k-NN

## 1. Convert Dense Ratings Matrix to Sparse User-Item Matrix

In [8]:
rating_df = pd.read_csv("../data/ratings.csv")

In [10]:
rating_df.head()

,user,item,rating
0,1889878,CC0101EN,3.0
1,1342067,CL0101EN,3.0
2,1990814,ML0120ENv3,3.0
3,380098,BD0211EN,3.0
4,779563,DS0101EN,3.0


In [11]:
rating_sparse_df = rating_df.pivot(index='user', columns='item', values='rating').fillna(0).reset_index().rename_axis(index=None, columns=None)
rating_sparse_df.head()

,user,AI0111EN,BC0101EN,BC0201EN,BC0202EN,BD0101EN,BD0111EN,BD0115EN,BD0121EN,BD0123EN,...,SW0201EN,TA0105,TA0105EN,TA0106EN,TMP0101EN,TMP0105EN,TMP0106,TMP107,WA0101EN,WA0103EN
0,2,0.0,3.0,0.0,0.0,3.0,2.0,0.0,2.0,2.0,...,0.0,2.0,0.0,3.0,0.0,2.0,2.0,0.0,3.0,0.0
1,4,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,...,0.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0
2,5,2.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,...,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Item-Based Collaborative Filtering: k-NN with Suprise Library

In [15]:
from surprise import KNNBasic
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

In [16]:
# Surprise is a Recommender System Engine similar to Scikit-Learn
# https://surprise.readthedocs.io/en/stable/getting_started.html

In [18]:
# Suprise loads from file
rating_df.to_csv("../data/ratings_sparse.csv", index=False)
# Read the course rating dataset with columns user item rating
reader = Reader(
        line_format='user item rating', sep=',', skip_lines=1, rating_scale=(2, 3))

coruse_dataset = Dataset.load_from_file("../data/ratings_sparse.csv", reader=reader)

In [19]:
trainset, testset = train_test_split(coruse_dataset, test_size=.3)

In [20]:
print(f"Total {trainset.n_users} users and {trainset.n_items} items in the trainingset")

Total 31330 users and 125 items in the trainingset


In [21]:
# Define a KNNBasic() model
# more KNN model hyperparamets can be found here:
# https://surprise.readthedocs.io/en/stable/knn_inspired.html
sim_option = {
    'name': 'cosine',
    'user_based': False,
    #'user_based': True,
}

# Train the KNNBasic model on the trainset, and predict ratings for the testset
algo = KNNBasic(sim_options=sim_option)
algo.fit(trainset)

# Then compute RMSE
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.1939


0.19387109865857158

In [22]:
type(predictions)

list

In [23]:
# We could parse the Prediction objects to a dataframe
predictions[0]

Prediction(uid='993479', iid='BD0123EN', r_ui=3.0, est=3, details={'actual_k': 17, 'was_impossible': False})

## 3. User-Based Collaborative Filtering: k-NN Manually

In [24]:
## STEPS:
## - Calculate the similarity between two users using their rating history (the row vectors of interaction matrix)
## - Build a similarity matrix for each pair of users with the training dataset
## - For each user, find its k nearest neighbors in the sim matrix
## - For each rating in the test dataset, estimate its rating using the KNN collaborative filtering equations shown before
## - Calculate RMSE for the entire test dataset

In [25]:
from scipy.spatial.distance import cosine

In [26]:
# Extract user list
user_list = rating_sparse_df['user'].to_list()[:50] # only 50 users taken, otherwise it takes very long
user_list_df = pd.DataFrame(data=user_list, columns = ['user'])
# Empts similarity matrix
user_sim = np.zeros((len(user_list), len(user_list)))
# Compare users pairwise
for i, this_user in enumerate(user_list):
    this_user_ratings = rating_sparse_df[rating_sparse_df.user == this_user].iloc[:,1:].values
    for j, other_user in enumerate(user_list):
        other_user_ratings = rating_sparse_df[rating_sparse_df.user == other_user].iloc[:,1:].values
        similarity = 1 - cosine(this_user_ratings, other_user_ratings)
        # FIXME: matrix is symmetric, only half needs to be computed!
        user_sim[i,j] = similarity
# Assemble similarity dataframe
user_sim_df = pd.DataFrame(data=user_sim, columns=user_list)
user_sim_df = pd.concat([user_list_df, user_sim_df], axis=1)
# Set index to user to sort similar users easily later on
user_sim_df.set_index(user_sim_df['user'], inplace=True, drop=True)
user_sim_df.drop(['user'], axis=1, inplace=True)

In [27]:
user_sim_df.head(10)

,2,4,5,7,8,9,12,16,17,19,...,55,56,57,58,59,60,61,62,63,64
user,,,,,,,,,,,,,,,,,,,,,
2,1.000000,0.569970,0.546903,0.140028,0.188639,0.339486,0.339486,0.390850,0.093352,0.165025,...,0.198030,0.256718,0.140028,0.353156,0.140028,0.198030,0.585837,0.198030,0.427793,0.093352
4,0.569970,1.000000,0.353281,0.150756,0.261116,0.429058,0.333712,0.207514,0.150756,0.213201,...,0.213201,0.301511,0.150756,0.355280,0.150756,0.213201,0.826153,0.213201,0.657952,0.150756
5,0.546903,0.353281,1.000000,0.000000,0.075165,0.123508,0.247016,0.209072,0.000000,0.184115,...,0.092057,0.065094,0.130189,0.177627,0.130189,0.000000,0.299187,0.092057,0.255686,0.000000
7,0.140028,0.150756,0.000000,1.000000,0.000000,0.316228,0.316228,0.344124,0.000000,0.000000,...,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.176777,0.000000,0.218218,0.000000
8,0.188639,0.261116,0.075165,0.000000,1.000000,0.365148,0.182574,0.198680,0.000000,0.000000,...,0.000000,0.288675,0.000000,0.143223,0.000000,0.000000,0.306186,0.000000,0.251976,0.577350
9,0.339486,0.429058,0.123508,0.316228,0.365148,1.000000,0.300000,0.217643,0.000000,0.223607,...,0.223607,0.474342,0.316228,0.156893,0.316228,0.000000,0.447214,0.223607,0.345033,0.316228
12,0.339486,0.333712,0.247016,0.316228,0.182574,0.300000,1.000000,0.290191,0.000000,0.000000,...,0.000000,0.158114,0.000000,0.117670,0.000000,0.000000,0.335410,0.000000,0.345033,0.000000
16,0.390850,0.207514,0.209072,0.344124,0.198680,0.217643,0.290191,1.000000,0.000000,0.243332,...,0.243332,0.344124,0.344124,0.170733,0.344124,0.000000,0.243332,0.243332,0.225282,0.000000
17,0.093352,0.150756,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
# For user n=7, which are the closest k=3 users?
n = 7
u = user_list[n]
k = 3
user_sim_df[u].sort_values(ascending=False)[1:(k+1)] # first user is the user herself, discarded

user
2     0.390850
33    0.344124
40    0.344124
Name: 16, dtype: float64

In [29]:
# For user n=7, which is her ratings prediction using the k=3 closest neighbors?
n = 7
u = user_list[n]
k = 3
item_list = rating_sparse_df.columns[1:]
neighbor_users = list(user_sim_df[u].sort_values(ascending=False)[1:(k+1)].index)
neighbor_weights = list(user_sim_df[u].sort_values(ascending=False)[1:(k+1)].values)
estimared_ratings = [0]*len(item_list)
for i, item in enumerate(item_list):
    n = 0
    summed_weights = 0
    for j, user in enumerate(neighbor_users):
        r = rating_sparse_df.loc[rating_sparse_df.user==user, item].values[0]
        if r > 0:
            n += 1
            estimared_ratings[i] += r*neighbor_weights[j]
            summed_weights += neighbor_weights[j]
    if n > 0:
        estimared_ratings[i] /= summed_weights

In [30]:
true_ratings = rating_sparse_df.loc[rating_sparse_df.user==user].values[0,1:]

In [31]:
rmse = np.sqrt(np.sum(np.square(np.array(estimared_ratings)-true_ratings))/len(estimared_ratings))

In [32]:
rmse

1.8445474891761775

# Part 2: Non-Negative Matrix Factorization (NMF)

## 1. NMF with the Suprise Library

In [5]:
from surprise import NMF
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

In [6]:
ratings_df = pd.read_csv("../data/ratings.csv")

In [7]:
ratings_df.head()

,user,item,rating
0,1889878,CC0101EN,3.0
1,1342067,CL0101EN,3.0
2,1990814,ML0120ENv3,3.0
3,380098,BD0211EN,3.0
4,779563,DS0101EN,3.0


In [8]:
# Dense -> Sparse ratings
# HOWEVER: note that Suprise works with dense tables, too
ratings_sparse_df = ratings_df.pivot(index='user', columns='item', values='rating').fillna(0).reset_index().rename_axis(index=None, columns=None)
ratings_sparse_df.head()

,user,AI0111EN,BC0101EN,BC0201EN,BC0202EN,BD0101EN,BD0111EN,BD0115EN,BD0121EN,BD0123EN,...,SW0201EN,TA0105,TA0105EN,TA0106EN,TMP0101EN,TMP0105EN,TMP0106,TMP107,WA0101EN,WA0103EN
0,2,0.0,3.0,0.0,0.0,3.0,2.0,0.0,2.0,2.0,...,0.0,2.0,0.0,3.0,0.0,2.0,2.0,0.0,3.0,0.0
1,4,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,...,0.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0
2,5,2.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,...,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Read the course rating dataset with columns user item rating
# The CSV file can be a dense one
reader = Reader(
        line_format='user item rating', sep=',', skip_lines=1, rating_scale=(2, 3))
course_dataset = Dataset.load_from_file("../data/ratings.csv", reader=reader)

In [10]:
trainset, testset = train_test_split(course_dataset, test_size=.3)

In [11]:
print(f"Total {trainset.n_users} users and {trainset.n_items} items in the trainingset")

Total 31348 users and 126 items in the trainingset


In [12]:
algo = NMF(n_factors=15, n_epochs=50, verbose=True, random_state=123)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
RMSE: 0.21

0.21235480559689332

## 2. NMF with Scikit-Learn

In [13]:
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [14]:
# Scikit-learn uses the SPARSE representation
X_train, X_test = train_test_split(
    ratings_sparse_df.iloc[:,1:],
    test_size=0.3, # portion of dataset to allocate to test set
    random_state=42 # we are setting the seed here, ALWAYS DO IT!
)

In [15]:
nmf = NMF(n_components=15, init='random', random_state=818)
W = nmf.fit_transform(X_train) # (X.shape[0]=n_samples, n_components)
H = nmf.components_ # (n_components, X.shape[1]=n_features)
X_hat = W@H

In [16]:
W.shape

(23730, 15)

In [17]:
H.shape

(15, 126)

In [18]:
# The RMSE metric is worse, with a default configuration
print('RMSE: ', mean_squared_error(X_train, X_hat, squared=False))

RMSE:  0.3360953022513968


In [19]:
# IMPORTANT: The fitted NMF model has constant H components,
# but W is different for each input data X.
W_test = nmf.transform(X_test)

In [20]:
W_test.shape

(10171, 15)

In [21]:
X_test_hat = W_test@H

In [22]:
# The RMSE metric is worse, with a default configuration
print('RMSE: ', mean_squared_error(X_test, X_test_hat, squared=False))

RMSE:  0.3373680254871046
